In [2]:
import pandas as pd
import numpy as np
import requests
import json 
import time
import os
from itertools import islice
from itertools import product
import itertools
import re

key = "18699dc38df444605d15278bd4ddac0f" 
key = '5a91eee05bb200a412fdddf3f53778a8'

# Fetching Indonesian Statistics

The Indonesian Statistical Agency is <a href="https://www.bps.go.id"> Badan Pusat Statistik </a>. They have a key-gatekept api at <a href="https://webapi.bps.go.id/developer/"> webapi.bps.go.id/developer/ </a>

### Finding what data is available

Requesting tables requires a domain code. We care most about Indonesia as a whole. These can be fetched by making a call to: https://webapi.bps.go.id/v1/api/domain/type/all


In [4]:
domain_list_url = f'https://webapi.bps.go.id/v1/api/domain/type/all/key/{key}/'
req = requests.get(domain_list_url)

this returns us some json where the domains are located in 'data'

In [26]:
domains_json = json.loads(req.content)
for domain in domains_json['data'][1]:
    print(f" \033[1m {domain['domain_name']}\033[0m (id: {domain['domain_id']}, url: {domain['domain_url']})) ")


  Pusat (id: 0000, url: http://www.bps.go.id)) 
  Aceh (id: 1100, url: https://aceh.bps.go.id)) 
  Simeulue (id: 1101, url: https://simeuluekab.bps.go.id)) 
  Aceh Singkil (id: 1102, url: https://acehsingkilkab.bps.go.id)) 
  Aceh Selatan (id: 1103, url: https://acehselatankab.bps.go.id)) 
  Aceh Tenggara (id: 1104, url: https://acehtenggarakab.bps.go.id)) 
  Aceh Timur (id: 1105, url: https://acehtimurkab.bps.go.id)) 
  Aceh Tengah (id: 1106, url: https://acehtengahkab.bps.go.id)) 
  Aceh Barat (id: 1107, url: https://acehbaratkab.bps.go.id)) 
  Aceh Besar (id: 1108, url: https://acehbesarkab.bps.go.id)) 
  Pidie (id: 1109, url: https://pidiekab.bps.go.id)) 
  Bireuen (id: 1110, url: https://bireuenkab.bps.go.id)) 
  Aceh Utara (id: 1111, url: https://acehutarakab.bps.go.id)) 
  Aceh Barat Daya (id: 1112, url: https://acehbaratdayakab.bps.go.id)) 
  Gayo Lues (id: 1113, url: https://gayolueskab.bps.go.id)) 
  Aceh Tamiang (id: 1114, url: https://acehtamiangkab.bps.go.id)) 
  Nagan Ray

It looks like Pusat (Malay: center) is the one we want: it's code is 0 suggesting it is the root, and its url is the stat's agencies homepage.

### Finding what Data is available

BPS divides their data into dynamic tables and static tables. Let's first focus on dynamic tables:

Requesting from https://webapi.bps.go.id/v1/api/list/model/var will provide us with a list of all variables available in the dynamic tables for the root domain.

In [30]:
req_url = f"https://webapi.bps.go.id/v1/api/list/model/var/lang/eng/domain/0000/key/{key}/"
req = requests.get(req_url)
req_json = json.loads(req.content)
req_json['data'][0]

{'page': 1, 'pages': 131, 'per_page': 10, 'count': 10, 'total': 1305}

There are a lot! We'll need to make a new call for each page:

In [45]:
if os.path.exists('indonesia_0000_subject_responses.json'): # Load from file if it exists
    with open('indonesia_0000_subject_responses.json', 'r') as f:
        responses = json.load(f)
else:
    responses =  [json.loads(requests.get(f"https://webapi.bps.go.id/v1/api/list/model/var/lang/eng/domain/0000/page/1/key/{key}/").content)['data']]
    page = 1
    while page < responses[0][0]['pages']:
        page += 1
        #wait a bit
        time.sleep(0.25)
        req = requests.get(f"https://webapi.bps.go.id/v1/api/list/model/var/lang/eng/domain/0000/page/{page}/key/{key}/")
        response = json.loads(req.content)['data']
        print(f"Page {page} Fetched with status code {req.status_code}")
        responses.append(response)

let's cache this to a json to avoid making 131 calls next time and store in a df for easy access:

In [57]:
if not os.path.exists('indonesia_0000_subject_responses.json'): # Save to file if it doesn't exist
    with open('indonesia_0000_subject_responses.json', 'w') as f:
        json.dump(responses, f)
df = pd.concat([pd.DataFrame(res[1]) for res in responses])
df.to_csv("all_indonesian_series.csv", index=False)


trying a static table:

In [ ]:
req_url = f"https://webapi.bps.go.id/v1/api/list/model/var/lang/eng/domain/0000/key/{key}/"
req = requests.get(req_url)
req_json = json.loads(req.content)
req_json['data'][0]

### Fetching the data

Let's try first with inflation (code 0)

In [613]:
var_num = 1521 # 1521  1 is for inflation
vervar = 9999
domain = "0000"
req_url = f"https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/{domain}/var/{var_num}/key/{key}/"
req = requests.get(req_url)
res = json.loads(req.content)
print(res.keys())
print(f"turvar {res['turvar']}")
print(f"vervar {res['vervar']}")


dict_keys(['status', 'data-availability', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'metadata', 'datacontent'])
turvar [{'val': 191, 'label': 'Urban+Rural'}]
vervar [{'val': 9999, 'label': 'INDONESIA'}, {'val': 1100, 'label': 'ACEH'}, {'val': 1200, 'label': 'SUMATERA UTARA'}, {'val': 1300, 'label': 'SUMATERA BARAT'}, {'val': 1400, 'label': 'RIAU'}, {'val': 1500, 'label': 'JAMBI'}, {'val': 1600, 'label': 'SUMATERA SELATAN'}, {'val': 1700, 'label': 'BENGKULU'}, {'val': 1800, 'label': 'LAMPUNG'}, {'val': 1900, 'label': 'KEP. BANGKA BELITUNG'}, {'val': 2100, 'label': 'KEP. RIAU'}, {'val': 3100, 'label': 'DKI JAKARTA'}, {'val': 3200, 'label': 'JAWA BARAT'}, {'val': 3300, 'label': 'JAWA TENGAH'}, {'val': 3400, 'label': 'DI YOGYAKARTA'}, {'val': 3500, 'label': 'JAWA TIMUR'}, {'val': 3600, 'label': 'BANTEN'}, {'val': 5100, 'label': 'BALI'}, {'val': 5200, 'label': 'NUSA TENGGARA BARAT'}, {'val': 5300, 'label': 'NUSA TENGGARA TIMUR'}, {'val': 6100, 'label': 'KALIMANTAN BARAT

Looking at the returned datacontent, we can see that a dict is returned with keys identifying inflation values:

In [205]:
dict(islice(res['datacontent'].items(), 0, 5))

{'999910791': 2.58,
 '999910792': 2.34,
 '999910793': 0.55,
 '999910794': 3.02,
 '999910795': 3.05}

Unfortunately, in the data we get back, the keys are not dates, they're they're identifiers e.g. 999910791
Where:

- The first digits are the vervar (verticle variable?): region in this case; 9999 is Indonesia
- The next digit(s) are the variable: 1 is inflation
- The next are the year: 079 is 1979
- The last digit is the subyear measure (in this case, 1-12 are Jan-Dec, 13 is annual): 1 is January


### From Value Key to Dataframe:

Going from value key to date is a bit of a pain. Fields are not of a consistent length, and aren't padded with 0s. This means we can't just slice the string. Interpretting the string character by character is a nonstarter - there can be ambiguity as to whether the next character is a continuation of a field or start of a new field. An alternative approach is to generate all the keys we expect to see and then match them to the datacontent keys. This is what we'll do.

In [662]:
def get_df_backwards(res):
    """
    (request response content) -> (pandas dataframe)
    An attempt at parsing the keys by reading them backwards character by character and moving to the next key when the current key doesn't match any of the labels
    """
    turtahun_labels = {str(turtahun['val']):turtahun['label'] for turtahun in res['turtahun']}
    tahun_labels = {str(tahun['val']):tahun['label'] for tahun in res['tahun']}
    turvar_labels = {str(turvar['val']):turvar['label'] for turvar in res['turvar']}
    var_labels = {str(var['val']):var['label'] for var in res['var']}
    vervar_labels = {str(vervar['val']):vervar['label'] for vervar in res['vervar']}

    turtahuns = [] # Subyear Periods
    tahuns = [] # Years
    turvars = [] # Subvariables
    vars = [] # Variables
    vervars = [] # Subvariables

    for key in res['datacontent'].keys():
        try :
            full_key = key
            print("")
            print(f"key {key}")
            print(f"value {res['datacontent'][key]}")

            # We need to loop backwards through each key - why? Because fields aren't 0 padded and going forwards has edge cases where next character could be either vervar or var (e.g. 11 could be vervar 11 or leading in to 1 1521)
            turtahun_start = len(key)-1
            while(len( [turtahun for turtahun in turtahun_labels if turtahun.endswith(key[turtahun_start-1:])  ] ) > 0): # Whilst adding the previous character to the key still matches a turtahun add that character to the key
                turtahun_start -= 1
            turtahuns.append(turtahun_labels[key[turtahun_start:]])
            print(f"turtaun: key {key} turtahun {key[turtahun_start:]} label {turtahun_labels[key[turtahun_start:]]}")
            key = key[:turtahun_start]

            # Now do the same for tahun (years) 
            tahun_start = len(key)-1
            while(len( [tahun for tahun in tahun_labels if tahun.endswith(key[tahun_start-1:])  ] ) > 0): # Whilst adding the previous character to the key still matches a turtahun add that character to the key
                tahun_start -= 1
            tahuns.append(tahun_labels[key[tahun_start:]])
            print(f"tahun: key {key} tahun {key[tahun_start:]} label {tahun_labels[key[tahun_start:]]}")
            key = key[:tahun_start]

            # and for turvar (subvariables)
            turvar_start = len(key)-1
            while(len( [turvar for turvar in turvar_labels if turvar.endswith(key[turvar_start-1:])  ] ) > 0): # Whilst adding the previous character to the key still matches a turtahun add that character to the key
                turvar_start -= 1
            turvars.append(turvar_labels[key[turvar_start:]])
            print(f"turvar: key {key} turvar {key[turvar_start:]} label {turvar_labels[key[turvar_start:]]}")
            key = key[:turvar_start]

            # and for var (variables)
            var_start = len(key)-1
            while(len( [var for var in var_labels if var.endswith(key[var_start-1:])  ] ) > 0): # Whilst adding the previous character to the key still matches a turtahun add that character to the key
                var_start -= 1
            print(f"var: key {key} var {key[var_start:]} label {var_labels[key[var_start:]]}")
            vars.append(var_labels[key[var_start:]])
            key = key[:var_start]

            # and for vervar (subvariables)
            vervar_start = len(key)-1
            while (len( [vervar for vervar in vervar_labels if vervar.endswith(key[vervar_start-1:])] ) > 0) and vervar_start > 0: # Whilst adding the previous character to the key still matches a turtahun add that character to the key
                vervar_start -= 1

            vervars.append(vervar_labels[key[vervar_start:]])
            print(f"vervar: key {key} vervar {key[vervar_start:]} label {vervar_labels[key[vervar_start:]]}")
            key = key[:vervar_start]

            if len(key) > 0:
                raise Exception(f"Key not empty {key}")
        except Exception as e:
            # print red and bold
            print('\033[1;31m Error {e} \033[0m')
        
    df = pd.DataFrame({"year" : tahuns, "subyear" : turtahuns, "subvariable" : turvars, "variable" : vars, "vertical_variable" : vervars})
    return df

def get_df(res):
    """
    (request response content) -> (pandas dataframe)
    Parses bps response into a pandas dataframe by matching the keys to the labels
    """
    vervar_labels = {str(vervar['val']):vervar['label'] for vervar in res['vervar']}
    var_labels = {str(var['val']):var['label'] for var in res['var']}
    turvar_labels = {str(turvar['val']):turvar['label'] for turvar in res['turvar']}
    tahun_labels = {str(tahun['val']):tahun['label'] for tahun in res['tahun']}
    turtahun_labels = {str(turtahun['val']):turtahun['label'] for turtahun in res['turtahun']}

    possible_keys = {} # dict of key:{"year", "subyear", "subvariable", "variable", "vertical_variable"} # Cache all possible keys to link datacontent to
    for fields in itertools.product(vervar_labels, var_labels, turvar_labels, tahun_labels, turtahun_labels):
        key = "".join(fields)
        possible_keys[key] = {"year" : tahun_labels[fields[3]], "subyear" : turtahun_labels[fields[4]], "subvariable" : turvar_labels[fields[2]], "variable" : var_labels[fields[1]], "vertical_variable" : vervar_labels[fields[0]]}

    # Link the datacontent to the possible keys
    data = [dict(possible_keys[key], **{"value":value}) for key, value in res['datacontent'].items()]
    df = pd.DataFrame(data)
    return df


In [670]:
var_num = 2130 # 1521  1 is for inflation
vervar = 9999
domain = "0000"
req_url = f"https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/{domain}/var/{var_num}/key/{key}/"
print(f"req_url {req_url}")
req = requests.get(req_url)
res = json.loads(req.content)
print(res.keys())
print(f"turvar {res['turvar']}")
print(f"vervar {res['vervar']}")
""" print("")
print(f"tahun {res['tahun']}")
print(f"turtahun {res['turtahun']}") """

get_df(res)



req_url https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/2130/key/5a91eee05bb200a412fdddf3f53778a8/
dict_keys(['status', 'data-availability', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'metadata', 'datacontent'])
turvar [{'val': 0, 'label': 'Tidak Ada'}]
vervar [{'val': 100, 'label': '1. Household Consumption Expenditures'}, {'val': 110, 'label': 'a. Food and Beverages, Other Than Restaurants'}, {'val': 120, 'label': 'b. Clothing, Footwear and Maintenance Services'}, {'val': 130, 'label': 'c. Household Housing and Equipment'}, {'val': 140, 'label': 'd. Health and Education'}, {'val': 150, 'label': 'e. Transportation and Communication'}, {'val': 160, 'label': 'f. Restaurants and Hotels'}, {'val': 170, 'label': 'g. Others'}, {'val': 200, 'label': '2. Consumption Expenditures of Non-Profit Institutions that serve Households'}, {'val': 300, 'label': '3. Government Consumption Expenditures'}, {'val': 310, 'label': 'a. Collective Consumption'}, {'v

,year,subyear,subvariable,variable,vertical_variable,value
0,2011,Quarter 1,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,1. Household Consumption Expenditures,2.32
1,2011,Quarter II,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,1. Household Consumption Expenditures,2.42
2,2011,Quarter III,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,1. Household Consumption Expenditures,2.68
3,2011,Quarter IV,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,1. Household Consumption Expenditures,2.79
4,2012,Quarter 1,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,1. Household Consumption Expenditures,3.06
...,...,...,...,...,...,...
1675,2022,Quarter II,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,8. GROSS DOMESTIC PRODUCTS,5.25
1676,2022,Quarter III,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,8. GROSS DOMESTIC PRODUCTS,5.41
1677,2022,Quarter IV,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,8. GROSS DOMESTIC PRODUCTS,5.31
1678,2022,Annually,Tidak Ada,[2010 Version] 8. Source of Growth C to C Quar...,8. GROSS DOMESTIC PRODUCTS,5.31


In [671]:
res["turtahun"]

[{'val': 31, 'label': 'Quarter 1'},
 {'val': 32, 'label': 'Quarter II'},
 {'val': 33, 'label': 'Quarter III'},
 {'val': 34, 'label': 'Quarter IV'},
 {'val': 35, 'label': 'Annually'}]

### Parsing Dates
Records right now are identified by year and subyear. We want to convert this to a date. Before proceeding, we'll download and cache requests for relevant series (identified in Relevant_Parameters.csv).

In [611]:
params_df = pd.read_csv("Relevant_Parameters.csv", dtype=str)
params_df.dropna(inplace=True, subset=["Topic"])

In [628]:
def request_series(row):
    """
    (pandas dataframe row) -> (requests response content)
    """
    # if request content exists in Cached_Requests/ use that
    # else request content from bps and save to Cached_Requests/
    if os.path.exists(f"Cached_Requests/{row['Topic']}.json"):
        with open(f"Cached_Requests/{row['Topic']}.json", "r") as f:
            return(json.load(f))
    else:
        # Request content from bps
        req_url = f"https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/{row['Var']}/key/{key}/vervar/{row['Vervar']}/"
        if not pd.isna(row['Turvar']):
            req_url += f"turvar/{row['Turvar']}/"
        req = requests.get(req_url)
        time.sleep(5)
        res = json.loads(req.content)
        # Save request content to Cached_Requests/ if request was successful
        if "datacontent" in res.keys():
            with open(f"Cached_Requests/{row['Topic']}.json", "w") as f:
                json.dump(res, f)
        return res
data_requests = params_df.apply(request_series, axis=1)

Let's look at all the labels we have in Turtahun (subyear):

In [635]:
turtahun_lists = [res['turtahun'] for res in data_requests]
# Flatten this list of lists
turtahuns = [turtahun['label'] for turtahun_list in turtahun_lists for turtahun in turtahun_list]
print(set(turtahuns))

{'January', 'Quarter II', 'September', 'May', 'April', 'February', 'Annual', 'November', 'Semester 2 (September)', 'December', 'Semester 1 (March)', 'Quarter III', 'Yearly', 'March', 'Annually', 'Quarter 1', 'August', 'July', 'October', 'June', 'Quarter IV'}


There's nothing too surprising here - Just the months, quarters and a few semesters. Instead of any fancy parsing, we'll just use a dictionary to map these to their respective months.

In [642]:
def turtahun_month_day(turtahun):
    """
    (str) -> (str, str)
    Returns the month and day of the turtahun
    """
    turtahun_lookups = { "January": "01-01", "February": "02-01", "March": "03-01", "April": "04-01", "May": "05-01", "June": "06-01", "July": "07-01", "August": "08-01", "September": "09-01", "October": "10-01", "November": "11-01", "December": "12-01", "Quarter I": "01-01", "Quarter II": "04-01", "Quarter III": "07-01", "Quarter IV": "10-01", "Semester 1 (March)" : "03-01", "Semester 2 (September)" : "09-01", "Annual": "01-01", "Annually" : "01-01", "Yearly" : "01-01"}
    if turtahun in turtahun_lookups.keys():
        return turtahun_lookups[turtahun]
    else:
        #print in red and bold
        print('\033[1;31m {turtahun} not in turtahun_lookups, defaulting to 01-01 \033[0m')

now we can have a JS compliant date string:

In [640]:
df["date"] = df["year"]+"-"+df["subyear"].apply(turtahun_month_day)

### Reducing Down to 3 Functions
To automate this, let's have it in one function and avoid pandas. Steps:
1. Function to fetch data from api
2. Function to parse data

In [685]:
def turtahun_month_day(turtahun):
    """
    (str) -> (str, str)
    Returns the month and day of the turtahun
    """
    turtahun_lookups = { "January": "01-01", "February": "02-01", "March": "03-01", "April": "04-01", "May": "05-01", "June": "06-01", "July": "07-01", "August": "08-01", "September": "09-01", "October": "10-01", "November": "11-01", "December": "12-01", "Quarter I": "01-01", "Quarter 1": "01-01", "Quarter 2": "04-01", "Quarter II": "04-01", "Quarter III": "07-01", "Quarter 3": "07-01", "Quarter IV": "10-01","Quarter 4": "10-01","Semester 1 (March)" : "03-01", "Semester 2 (September)" : "09-01", "Annual": "01-01", "Annually" : "01-01", "Yearly" : "01-01"}
    if turtahun in turtahun_lookups:
        return turtahun_lookups[turtahun]
    else:
        #print in red and bold
        print(f'\033[1;31m {turtahun} not in turtahun_lookups, defaulting to 01-01 \033[0m')
        return "01-01"

def dict_from_req(res):
    """
    (requests response) -> (json: list of dicts of form {"date":str, "value":str}})
    Receives a requests response of a bps api request and json containing the datacontent
    """
    vervar_labels = {str(vervar['val']):vervar['label'] for vervar in res['vervar']}
    var_labels = {str(var['val']):var['label'] for var in res['var']}
    turvar_labels = {str(turvar['val']):turvar['label'] for turvar in res['turvar']}
    tahun_labels = {str(tahun['val']):tahun['label'] for tahun in res['tahun']}
    turtahun_labels = {str(turtahun['val']):turtahun['label'] for turtahun in res['turtahun']}

    possible_keys = {} # dict of key:{"year", "subyear", "subvariable", "variable", "vertical_variable"} # Cache all possible keys to link datacontent to
    for fields in itertools.product(vervar_labels, var_labels, turvar_labels, tahun_labels, turtahun_labels):
        key = "".join(fields)
        possible_keys[key] = {"year" : tahun_labels[fields[3]], "subyear" : turtahun_labels[fields[4]], "subvariable" : turvar_labels[fields[2]], "variable" : var_labels[fields[1]], "vertical_variable" : vervar_labels[fields[0]]}
    data = [dict(possible_keys[key], **{"value":value}) for key, value in res['datacontent'].items()]

    for d in data:
        #print(f"d['year'] = {d['year']}, d['subyear'] = {d['subyear']}")
        #print(f" type(d['year']) = {type(d['year'])}, type(d['subyear']) = {type(d['subyear'])}, type(turtahun_month_day(d['subyear'])) = {type(turtahun_month_day(d['subyear']))}")
        d["date"] = d["year"]+"-"+turtahun_month_day(d["subyear"])

    return {"name":res['var'][0]['label'],
             "unit": res['var'][0]['unit'],
             "data":data }


def request_bps_series_from_csv(series_path="Relevant_Parameters.csv", write_path="bps_data/"):
    """
    (path of csv containing series to get) -> (none)
    Requests data from bps and saves it to write_path
    """
    params_df = pd.read_csv("Relevant_Parameters.csv", dtype=str)
    params_df.dropna(inplace=True, subset=["Topic"])

    for index, row in params_df.iterrows():
        req_url = f"https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/{row['Var']}/key/{key}/vervar/{row['Vervar']}/"
        if not pd.isna(row['Turvar']):
            req_url += f"turvar/{row['Turvar']}/"
        req = requests.get(req_url)
        time.sleep(1)
        res = json.loads(req.content)
        if "datacontent" in res.keys():
            with open(f"{write_path}{row['Topic']}.json", "w") as f:
                json.dump(dict_from_req(res), f)
            print(f"Finished {row['Topic']} with status code {req.status_code}")
        else:
            print(f"Failed {row['Topic']} with status code {req.status_code}")
    

In [3]:
import pandas as pd

params_df = pd.read_csv("Relevant_Parameters.csv", dtype=str)
params_df.dropna(inplace=True, subset=["Topic"])



for index, row in params_df.iterrows():
    req_url = f"https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/{row['Var']}/key/{key}/vervar/{row['Vervar']}/"
    if not pd.isna(row['Turvar']):
        req_url += f"turvar/{row['Turvar']}/"
    print(req_url)
    


https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/1/key/5a91eee05bb200a412fdddf3f53778a8/vervar/9999/
https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/2130/key/5a91eee05bb200a412fdddf3f53778a8/vervar/800/
https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/543/key/5a91eee05bb200a412fdddf3f53778a8/vervar/9999/
https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/1953/key/5a91eee05bb200a412fdddf3f53778a8/vervar/1/turvar/1664/
https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/1953/key/5a91eee05bb200a412fdddf3f53778a8/vervar/2/turvar/1664/
https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/196/key/5a91eee05bb200a412fdddf3f53778a8/vervar/9999/
https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/497/key/5a91eee05bb200a412fdddf3f53778a8/vervar/9999/
https://webapi.bps.go.id/v1/api/list/model/data/lang/eng/domain/0000/var/720/key/5a91eee05bb2

In [684]:
request_bps_series_from_csv()

Finished Inflation  with status code 200
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
 Quarter 1 not in turtahun_lookups, defaulting to 01-01 
Finished Growth  with status code 200
Finished Unemployment rate  with status code 200
Finished Employment rate with status code 200
Finished Employed with status code 200
Finished Exports with status code 200
Finished

In [679]:
turtahun_month_day("Quarter I")

'01-01'

In [623]:
data_requests

NameError: name 'data_requests' is not defined

In [617]:
params_df.iloc[0].Turvar == nan

AttributeError: 'float' object has no attribute 'isnan'

In [543]:
key = "115210115189"

In [544]:
len("115210115189")

12

In [545]:
turtahun_start = len(key)-1
turtahun_start

11

In [546]:
key[turtahun_start-1]

'8'

In [ ]:
while(len( [turtahun for turtahun in turtahun_labels if turtahun.endswith(key[turtahun_start-1])  ] ) > 0): # Whilst adding the previous character to the key still matches a turtahun add that character to the key
    turtahun_start -= 1
turtahuns.append(turtahun_labels[key[turtahun_start:]])
key = key[:turtahun_start]


In [519]:
res['turtahun']

[{'val': 189, 'label': 'February'}, {'val': 190, 'label': 'August'}]

In [517]:
"hello"[len("hello"):]

''

In [505]:
[key for key in res['datacontent'].keys() if key.startswith("11") ]

['115210115189',
 '115210115190',
 '115210116189',
 '115210116190',
 '115210117189',
 '115210117190',
 '115210118189',
 '115210118190',
 '115210119189',
 '115210119190',
 '115210120189',
 '115210120190',
 '115210121189',
 '115210121190',
 '115210122189',
 '115210122190',
 '115210123189',
 '1115210115189',
 '1115210115190',
 '1115210116189',
 '1115210116190',
 '1115210117189',
 '1115210117190',
 '1115210118189',
 '1115210118190',
 '1115210119189',
 '1115210119190',
 '1115210120189',
 '1115210120190',
 '1115210121189',
 '1115210121190',
 '1115210122189',
 '1115210122190',
 '1115210123189']

In [503]:
def data_from_bps(res):
    """"
    req: requests content
    Returns: pandas dataframe of the data
    """

    vervar_len = max([len(str(vervar['val'])) for vervar in res['vervar']]) # max length of vervar
    vervar_labels = {str(vervar['val']): vervar['label'] for vervar in res['vervar']} # associate vervar with labels
    vertical_variables = []

    turvar_len = max([len(str(turvar['val'])) for turvar in res['turvar']])
    turvar_labels = {str(turvar['val']): turvar['label'] for turvar in res['turvar']} # associate turvar with labels
    turvar_labels['Tidak Ada'] = ''
    subvariables = []

    var_len = len(str(res['var'][0]['val']))
    var_labels = {str(var['val']): var['label'] for var in res['var']} # associate var with labels
    variables = []

    #tahun_len = max([len(str(tahun['val'])) for tahun in res['tahun']]) 
    tahun_labels = {str(tahun['val']): tahun['label'] for tahun in res['tahun']} # associate tahun with labels - they don't zfill at all - they just add a 0 regardless of length!!
    years = []

    turtahun_len = max([len(str(turtahun['val'])) for turtahun in res['turtahun']])
    turtahun_labels = {str(turtahun['val']): turtahun['label'] for turtahun in res['turtahun']} # associate tahun with labels - they do zfill
    subyears = []

    keys = []
    values = []

    print(f"res['vervar']: {res['vervar']}")
    print(f"res['var']: {res['var']}")
    print("")

    for key in res['datacontent'].keys():

        keys.append(key)
        full_key = key


        values.append(
            res['datacontent'][key]
        )

        # First extract the vervar and var - these aren't 0 separated
        """     print(f"key: {key}")
        print(f"full_key: {full_key}")
        print(f"vervar_len: {vervar_len}") 
        """

        # This isn't working - turns out they don't actually zfill
        # TODO: grab next characters until it couldn't be a vervar

        vervar_len = 1
        while len([vervar for vervar in res['vervar'] if str(vervar['val']).startswith(key[0:vervar_len+1])]) > 0:
            vervar_len += 1

        vertical_variables.append(
            vervar_labels[
                key[0:vervar_len]
            ]
        )
        key = key[vervar_len:]

        var_len = 1
        print(f"full_key: {full_key}")
        print(f"key: {key}")
        print(f"key starts with {key[0:var_len]}")
        print(f"variables starting with {key[0:var_len]}: {[var for var in res['var'] if str(var['val']).startswith(key[0:var_len+1])]}")
        while len([var for var in res['var'] if str(var['val']).startswith(key[0:var_len+1])]) > 0:
            var_len += 1

        print(f"res['var']: {res['var']}")
        print(f"var_len: {var_len}")
        print(f"var: {key[0:var_len]}")
        print(f"var_labels: {var_labels}")


        variables.append(
            var_labels[
                key[0:var_len]
            ]
        )
        key = key[var_len:]

        # Then extract turvar
        subvariables.append(
            turvar_labels[
                key[0:turvar_len]
            ]
        )
        key = key[turvar_len:]

        # Work out how many digits tahun will have
        # look at the first character and find maximum length of tahuns that start with that character
        tahun_len = max([ len(str(tahun['val'])) for tahun in res['tahun'] if str(tahun['val']).startswith(key[0])])
        """ print(f"key: {key}")
        print(f"full_key: {full_key}")
        print(f"tahun_len: {tahun_len}")
        print(f"tahun starts with {key[0]}")
        print(f"keys starting with {key[0]} are {[ str(tahun['val']) for tahun in res['tahun'] if str(tahun['val']).startswith(key[0])]}") """
        years.append(
            tahun_labels[
                key[0:tahun_len]
            ]
        )
        key = key[tahun_len:]

        # Which only leaves the subyear
        subyears.append(
            turtahun_labels[
                key
            ]
        )
        # When there are varying lengths of turhan, they are separated by a 0
         
    return pd.DataFrame({
        "vertical_variable": vertical_variables,
        "variable": variables,
        "subvariable": subvariables,
        "year": years,
        "subyear": subyears,
        "key": keys,
        "value": values
    })

df = data_from_bps(res)
df
    

res['vervar']: [{'val': 1, 'label': 'A'}, {'val': 2, 'label': 'B'}, {'val': 3, 'label': 'C'}, {'val': 4, 'label': 'D'}, {'val': 5, 'label': 'E'}, {'val': 6, 'label': 'F'}, {'val': 7, 'label': 'G'}, {'val': 8, 'label': 'H'}, {'val': 9, 'label': 'I'}, {'val': 10, 'label': 'J'}, {'val': 11, 'label': 'K'}, {'val': 12, 'label': 'L'}, {'val': 13, 'label': 'M, N'}, {'val': 14, 'label': 'O'}, {'val': 15, 'label': 'P'}, {'val': 16, 'label': 'Q'}, {'val': 17, 'label': 'R, S, T, U'}, {'val': 18, 'label': 'Average'}]
res['var']: [{'val': 1521, 'label': 'Average of Net Wage/Salary', 'unit': 'Rupiahs', 'subj': 'Labour Wages', 'def': '', 'decimal': '0', 'note': 'Source from The  Labor  Force  Situation  in Indonesia February and August 2023.\n Net  Wage/Salary  per month is  wage/salary received  by  the  employee  during  previous  month  from  the  main  job,which consists of basic wages and allowances, in terms of money or goods paid by the employer.\nA. Agriculture, Forestry, and Fishing   \t\t\t

KeyError: '5'

In [464]:
df.query("key == '119531664111189'")

,vertical_variable,variable,subvariable,year,subyear,key,value
0,Percentage (%),Number and Percentage of Employment and Unempl...,Employment,2011,February,119531664111189,93.04


In [467]:
res['turtahun']

[{'val': 189, 'label': 'February'}, {'val': 190, 'label': 'August'}]

In [431]:
res['turvar']

[{'val': 0, 'label': 'Tidak Ada'}]

In [423]:
[ [key, value] for key, value in res['datacontent'].items() if key == "119531664111189"]

[['119531664111189', 93.04]]

In [424]:
res['turtahun']

[{'val': 189, 'label': 'February'}, {'val': 190, 'label': 'August'}]

In [384]:
res

{'status': 'OK',
 'data-availability': 'available',
 'var': [{'val': 1,
   'label': 'Inflation (Umum)',
   'unit': '',
   'subj': 'Consumer Prices Indices',
   'def': '',
   'decimal': '',
   'note': '&lt;p&gt;1. Before April 1979, the base year is September 1966 (September 1966 = 100)&lt;br /&gt;\n2.Since April 1979, the Term &quot;Consumer Price Index&quot; has been used ( the term that has been used before is &quot;Cost Living Index&quot;). The base year is April 1977-Maret 1978. CPI using a consumption pattern obtained from 1977/1978 Cost Living Survey in 17 Provincial Capital cities.) (April 1977-Maret 1978=100).&lt;br /&gt;\n3. Since April 1990-1997, CPI has been based on a consumption pattern obtained from Cost Living Survey in 27 Provincial Capital cities and using 1988/1989 base year) (1988/1989=100)&lt;br /&gt;\n4. Since December 1997, CPI has been based on a consumption pattern obtained from 1996 Cost of Living Survey in 44 cities(1996=100)&lt;br /&gt;\n5. Since January 2004

In [ ]:
res['']

In [375]:
res['tahun']

[{'val': 79, 'label': '1979'},
 {'val': 80, 'label': '1980'},
 {'val': 81, 'label': '1981'},
 {'val': 82, 'label': '1982'},
 {'val': 83, 'label': '1983'},
 {'val': 84, 'label': '1984'},
 {'val': 85, 'label': '1985'},
 {'val': 86, 'label': '1986'},
 {'val': 87, 'label': '1987'},
 {'val': 88, 'label': '1988'},
 {'val': 89, 'label': '1989'},
 {'val': 90, 'label': '1990'},
 {'val': 91, 'label': '1991'},
 {'val': 92, 'label': '1992'},
 {'val': 93, 'label': '1993'},
 {'val': 94, 'label': '1994'},
 {'val': 95, 'label': '1995'},
 {'val': 96, 'label': '1996'},
 {'val': 97, 'label': '1997'},
 {'val': 98, 'label': '1998'},
 {'val': 99, 'label': '1999'},
 {'val': 100, 'label': '2000'},
 {'val': 101, 'label': '2001'},
 {'val': 102, 'label': '2002'},
 {'val': 103, 'label': '2003'},
 {'val': 104, 'label': '2004'},
 {'val': 105, 'label': '2005'},
 {'val': 106, 'label': '2006'},
 {'val': 107, 'label': '2007'},
 {'val': 108, 'label': '2008'},
 {'val': 109, 'label': '2009'},
 {'val': 110, 'label': '2010'

In [347]:
res['tahun']

[{'val': 79, 'label': '1979'},
 {'val': 80, 'label': '1980'},
 {'val': 81, 'label': '1981'},
 {'val': 82, 'label': '1982'},
 {'val': 83, 'label': '1983'},
 {'val': 84, 'label': '1984'},
 {'val': 85, 'label': '1985'},
 {'val': 86, 'label': '1986'},
 {'val': 87, 'label': '1987'},
 {'val': 88, 'label': '1988'},
 {'val': 89, 'label': '1989'},
 {'val': 90, 'label': '1990'},
 {'val': 91, 'label': '1991'},
 {'val': 92, 'label': '1992'},
 {'val': 93, 'label': '1993'},
 {'val': 94, 'label': '1994'},
 {'val': 95, 'label': '1995'},
 {'val': 96, 'label': '1996'},
 {'val': 97, 'label': '1997'},
 {'val': 98, 'label': '1998'},
 {'val': 99, 'label': '1999'},
 {'val': 100, 'label': '2000'},
 {'val': 101, 'label': '2001'},
 {'val': 102, 'label': '2002'},
 {'val': 103, 'label': '2003'},
 {'val': 104, 'label': '2004'},
 {'val': 105, 'label': '2005'},
 {'val': 106, 'label': '2006'},
 {'val': 107, 'label': '2007'},
 {'val': 108, 'label': '2008'},
 {'val': 109, 'label': '2009'},
 {'val': 110, 'label': '2010'

In [343]:
"010 0 11".split('0', 2)

['', '1', ' 0 11']

In [317]:
d = {'a': 1, 'b': 2, 'c': 3}

In [344]:
res['datacontent'].keys() 

dict_keys(['999910791', '999910792', '999910793', '999910794', '999910795', '999910796', '999910797', '999910798', '999910799', '9999107910', '9999107911', '9999107912', '9999107913', '999910801', '999910802', '999910803', '999910804', '999910805', '999910806', '999910807', '999910808', '999910809', '9999108010', '9999108011', '9999108012', '9999108013', '999910811', '999910812', '999910813', '999910814', '999910815', '999910816', '999910817', '999910818', '999910819', '9999108110', '9999108111', '9999108112', '9999108113', '999910821', '999910822', '999910823', '999910824', '999910825', '999910826', '999910827', '999910828', '999910829', '9999108210', '9999108211', '9999108212', '9999108213', '999910831', '999910832', '999910833', '999910834', '999910835', '999910836', '999910837', '999910838', '999910839', '9999108310', '9999108311', '9999108312', '9999108313', '999910841', '999910842', '999910843', '999910844', '999910845', '999910846', '999910847', '999910848', '999910849', '999910

In [371]:
[f'{key[:6]} {key[6:]}' for key in res['datacontent'].keys() if key[0:4] == '9999' and (key[6:8]=="99" or key[6:9]=="100") and '999910' in key]

['999910 991',
 '999910 992',
 '999910 993',
 '999910 994',
 '999910 995',
 '999910 996',
 '999910 997',
 '999910 998',
 '999910 999',
 '999910 9910',
 '999910 9911',
 '999910 9912',
 '999910 9913',
 '999910 1001',
 '999910 1002',
 '999910 1003',
 '999910 1004',
 '999910 1005',
 '999910 1006',
 '999910 1007',
 '999910 1008',
 '999910 1009',
 '999910 10010',
 '999910 10011',
 '999910 10012',
 '999910 10013']

In [300]:
json.dump(res, open("indonesia_0000_543.json", "w"))

In [301]:
res['var']

[{'val': 543,
  'label': 'Unemployment Rate by Province',
  'unit': 'Percent',
  'subj': 'Employment',
  'def': '',
  'decimal': '2',
  'note': '&lt;pre class=&quot;tw-data-text tw-text-large XcVN5d tw-ta&quot; style=&quot;text-align:left;&quot; dir=&quot;ltr&quot;&gt;&lt;span lang=&quot;en&quot; xml:lang=&quot;en&quot;&gt;The 2011-2013 data uses the Backcast of the Component Projection Weigher\nThe 2018-2020 data uses Backcast from the 2015 SUPAS-based Weigher\n1986-1998, calculations without data for the Province of Timor Timor\nIn 1995, Sakernas was not implemented\nIn 2000, without Maluku\nSource: National Labor Force Survey (Sakernas)&lt;/span&gt;&lt;/pre&gt;&lt;p&gt;&lt;br /&gt;&lt;/p&gt;&lt;br /&gt;&lt;p&gt;&lt;br /&gt;&lt;/p&gt;&lt;p&gt;&lt;br /&gt;&lt;/p&gt;'}]

In [345]:
list(res['datacontent'].keys() )

['999910791',
 '999910792',
 '999910793',
 '999910794',
 '999910795',
 '999910796',
 '999910797',
 '999910798',
 '999910799',
 '9999107910',
 '9999107911',
 '9999107912',
 '9999107913',
 '999910801',
 '999910802',
 '999910803',
 '999910804',
 '999910805',
 '999910806',
 '999910807',
 '999910808',
 '999910809',
 '9999108010',
 '9999108011',
 '9999108012',
 '9999108013',
 '999910811',
 '999910812',
 '999910813',
 '999910814',
 '999910815',
 '999910816',
 '999910817',
 '999910818',
 '999910819',
 '9999108110',
 '9999108111',
 '9999108112',
 '9999108113',
 '999910821',
 '999910822',
 '999910823',
 '999910824',
 '999910825',
 '999910826',
 '999910827',
 '999910828',
 '999910829',
 '9999108210',
 '9999108211',
 '9999108212',
 '9999108213',
 '999910831',
 '999910832',
 '999910833',
 '999910834',
 '999910835',
 '999910836',
 '999910837',
 '999910838',
 '999910839',
 '9999108310',
 '9999108311',
 '9999108312',
 '9999108313',
 '999910841',
 '999910842',
 '999910843',
 '999910844',
 '999910845',


In [295]:
data_from_bps(res)

{'086': '1986', '087': '1987', '088': '1988', '089': '1989', '090': '1990', '091': '1991', '092': '1992', '093': '1993', '094': '1994', '096': '1996', '097': '1997', '098': '1998', '099': '1999', '100': '2000', '101': '2001', '102': '2002', '103': '2003', '104': '2004', '105': '2005', '106': '2006', '107': '2007', '108': '2008', '109': '2009', '110': '2010', '111': '2011', '112': '2012', '113': '2013', '114': '2014', '115': '2015', '116': '2016', '117': '2017', '118': '2018', '119': '2019', '120': '2020', '121': '2021', '122': '2022', '123': '2023'}
['086', '087', '088', '089', '090', '091', '092', '093', '094', '096', '097', '098', '099', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '011', '012', '012', '012', '012', '012', '012', '012', '086', '087', '088', '089', '090', '091', '092', '093', '0

KeyError: '010'

In [287]:
len(str(res['var'][0]['val']))


3

In [283]:
res.keys()

dict_keys(['status', 'data-availability', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'metadata', 'datacontent'])

In [280]:
'12'.zfill(4)

'0012'

In [281]:
data_from_bps(res)

['INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'INDONESIA',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACEH',
 'ACE

In [ ]:
# reshape res['vervar] from list of dicts to dict of dicts
vervar_dict = {

In [265]:
vervar_dict = {vervar['val']: vervar['label'] for vervar in res['vervar']}
vervar_dict

{9999: 'INDONESIA',
 1100: 'ACEH',
 1200: 'SUMATERA UTARA',
 1300: 'SUMATERA BARAT',
 1400: 'RIAU',
 1500: 'JAMBI',
 1600: 'SUMATERA SELATAN',
 1700: 'BENGKULU',
 1800: 'LAMPUNG',
 1900: 'KEP. BANGKA BELITUNG',
 2100: 'KEP. RIAU',
 3100: 'DKI JAKARTA',
 3200: 'JAWA BARAT',
 3300: 'JAWA TENGAH',
 3400: 'DI YOGYAKARTA',
 3500: 'JAWA TIMUR',
 3600: 'BANTEN',
 5100: 'BALI',
 5200: 'NUSA TENGGARA BARAT',
 5300: 'NUSA TENGGARA TIMUR',
 6100: 'KALIMANTAN BARAT',
 6200: 'KALIMANTAN TENGAH',
 6300: 'KALIMANTAN SELATAN',
 6400: 'KALIMANTAN TIMUR',
 6500: 'KALIMANTAN UTARA',
 7100: 'SULAWESI UTARA',
 7200: 'SULAWESI TENGAH',
 7300: 'SULAWESI SELATAN',
 7400: 'SULAWESI TENGGARA',
 7500: 'GORONTALO',
 7600: 'SULAWESI BARAT',
 8100: 'MALUKU',
 8200: 'MALUKU UTARA',
 9100: 'PAPUA BARAT',
 9400: 'PAPUA'}

In [259]:
res['datacontent'].items()

dict_items([('9999543086191', 2.7), ('9999543087191', 2.62), ('9999543088191', 2.85), ('9999543089191', 2.81), ('9999543090191', 2.55), ('9999543091191', 2.62), ('9999543092191', 2.74), ('9999543093191', 2.79), ('9999543094191', 4.36), ('9999543096191', 4.87), ('9999543097191', 4.69), ('9999543098191', 5.46), ('9999543099191', 6.36), ('99995430100191', 6.08), ('99995430101191', 8.1), ('99995430102191', 9.06), ('99995430103191', 9.67), ('99995430104191', 9.86), ('99995430105189', 10.26), ('99995430105190', 11.24), ('99995430106189', 10.45), ('99995430106190', 10.28), ('99995430107189', 9.75), ('99995430107190', 9.11), ('99995430108189', 8.46), ('99995430108190', 8.39), ('99995430109189', 8.14), ('99995430109190', 7.87), ('99995430110189', 7.41), ('99995430110190', 7.14), ('99995430111189', 6.96), ('99995430111190', 7.48), ('99995430112189', 6.37), ('99995430112190', 6.13), ('99995430113189', 5.88), ('99995430113190', 6.17), ('99995430114189', 5.7), ('99995430114190', 5.94), ('9999543011

In [242]:
[key for key in res['datacontent'].keys() if "99995430110" in key]

['99995430110189', '99995430110190']

In [236]:
len("9999543110")

10

In [217]:
len("t")

10

In [244]:
res['turtahun']

[{'val': 191, 'label': 'Yearly'},
 {'val': 189, 'label': 'February'},
 {'val': 190, 'label': 'August'}]

10791

In [ ]:
res['datacontent']['9999']

In [185]:
res['turtahun']

[{'val': 1, 'label': 'January'},
 {'val': 2, 'label': 'February'},
 {'val': 3, 'label': 'March'},
 {'val': 4, 'label': 'April'},
 {'val': 5, 'label': 'May'},
 {'val': 6, 'label': 'June'},
 {'val': 7, 'label': 'July'},
 {'val': 8, 'label': 'August'},
 {'val': 9, 'label': 'September'},
 {'val': 10, 'label': 'October'},
 {'val': 11, 'label': 'November'},
 {'val': 12, 'label': 'December'},
 {'val': 13, 'label': 'Annually'}]

In [182]:
res['datacontent']

{'999910791': 2.58,
 '999910792': 2.34,
 '999910793': 0.55,
 '999910794': 3.02,
 '999910795': 3.05,
 '999910796': 2.32,
 '999910797': 2.5,
 '999910798': 2.34,
 '999910799': 0.74,
 '9999107910': 0.89,
 '9999107911': 0.57,
 '9999107912': 0.87,
 '9999107913': 21.77,
 '999910801': 1.19,
 '999910802': 1.42,
 '999910803': 0.22,
 '999910804': 1.04,
 '999910805': 3.79,
 '999910806': 1.49,
 '999910807': 1.1,
 '999910808': 1.19,
 '999910809': 0.36,
 '9999108010': 1.69,
 '9999108011': 2.22,
 '9999108012': 0.26,
 '9999108013': 15.97,
 '999910811': 1.31,
 '999910812': 0.7,
 '999910813': 0.7,
 '999910814': 0.93,
 '999910815': 0.15,
 '999910816': 0.42,
 '999910817': 1.2,
 '999910818': 0.51,
 '999910819': -0.19,
 '9999108110': 1.17,
 '9999108111': -0.32,
 '9999108112': 0.51,
 '9999108113': 7.09,
 '999910821': 4.7,
 '999910822': 0.54,
 '999910823': 0.18,
 '999910824': -0.06,
 '999910825': 0.18,
 '999910826': 0.33,
 '999910827': 1.11,
 '999910828': -0.46,
 '999910829': 0.88,
 '9999108210': 1.24,
 '99991

In [184]:
res['tahun']

[{'val': 79, 'label': '1979'},
 {'val': 80, 'label': '1980'},
 {'val': 81, 'label': '1981'},
 {'val': 82, 'label': '1982'},
 {'val': 83, 'label': '1983'},
 {'val': 84, 'label': '1984'},
 {'val': 85, 'label': '1985'},
 {'val': 86, 'label': '1986'},
 {'val': 87, 'label': '1987'},
 {'val': 88, 'label': '1988'},
 {'val': 89, 'label': '1989'},
 {'val': 90, 'label': '1990'},
 {'val': 91, 'label': '1991'},
 {'val': 92, 'label': '1992'},
 {'val': 93, 'label': '1993'},
 {'val': 94, 'label': '1994'},
 {'val': 95, 'label': '1995'},
 {'val': 96, 'label': '1996'},
 {'val': 97, 'label': '1997'},
 {'val': 98, 'label': '1998'},
 {'val': 99, 'label': '1999'},
 {'val': 100, 'label': '2000'},
 {'val': 101, 'label': '2001'},
 {'val': 102, 'label': '2002'},
 {'val': 103, 'label': '2003'},
 {'val': 104, 'label': '2004'},
 {'val': 105, 'label': '2005'},
 {'val': 106, 'label': '2006'},
 {'val': 107, 'label': '2007'},
 {'val': 108, 'label': '2008'},
 {'val': 109, 'label': '2009'},
 {'val': 110, 'label': '2010'

In [175]:
res.keys()

dict_keys(['status', 'data-availability', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'metadata', 'datacontent'])

In [176]:
res['var']

[{'val': 1,
  'label': 'Inflation (Umum)',
  'unit': '',
  'subj': 'Consumer Prices Indices',
  'def': '',
  'decimal': '',
  'note': '&lt;p&gt;1. Before April 1979, the base year is September 1966 (September 1966 = 100)&lt;br /&gt;\n2.Since April 1979, the Term &quot;Consumer Price Index&quot; has been used ( the term that has been used before is &quot;Cost Living Index&quot;). The base year is April 1977-Maret 1978. CPI using a consumption pattern obtained from 1977/1978 Cost Living Survey in 17 Provincial Capital cities.) (April 1977-Maret 1978=100).&lt;br /&gt;\n3. Since April 1990-1997, CPI has been based on a consumption pattern obtained from Cost Living Survey in 27 Provincial Capital cities and using 1988/1989 base year) (1988/1989=100)&lt;br /&gt;\n4. Since December 1997, CPI has been based on a consumption pattern obtained from 1996 Cost of Living Survey in 44 cities(1996=100)&lt;br /&gt;\n5. Since January 2004, CPI has been based on a consumption pattern obtained from 2002 C

In [183]:
res['tahun']

[{'val': 79, 'label': '1979'},
 {'val': 80, 'label': '1980'},
 {'val': 81, 'label': '1981'},
 {'val': 82, 'label': '1982'},
 {'val': 83, 'label': '1983'},
 {'val': 84, 'label': '1984'},
 {'val': 85, 'label': '1985'},
 {'val': 86, 'label': '1986'},
 {'val': 87, 'label': '1987'},
 {'val': 88, 'label': '1988'},
 {'val': 89, 'label': '1989'},
 {'val': 90, 'label': '1990'},
 {'val': 91, 'label': '1991'},
 {'val': 92, 'label': '1992'},
 {'val': 93, 'label': '1993'},
 {'val': 94, 'label': '1994'},
 {'val': 95, 'label': '1995'},
 {'val': 96, 'label': '1996'},
 {'val': 97, 'label': '1997'},
 {'val': 98, 'label': '1998'},
 {'val': 99, 'label': '1999'},
 {'val': 100, 'label': '2000'},
 {'val': 101, 'label': '2001'},
 {'val': 102, 'label': '2002'},
 {'val': 103, 'label': '2003'},
 {'val': 104, 'label': '2004'},
 {'val': 105, 'label': '2005'},
 {'val': 106, 'label': '2006'},
 {'val': 107, 'label': '2007'},
 {'val': 108, 'label': '2008'},
 {'val': 109, 'label': '2009'},
 {'val': 110, 'label': '2010'

In [180]:
[yr for yr in res['tahun'] if yr['val']==107]

[{'val': 107, 'label': '2007'}]

In [163]:
print(res.keys())
res["datacontent"]

dict_keys(['status', 'data-availability', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'metadata', 'datacontent'])


{'999910791': 2.58,
 '999910792': 2.34,
 '999910793': 0.55,
 '999910794': 3.02,
 '999910795': 3.05,
 '999910796': 2.32,
 '999910797': 2.5,
 '999910798': 2.34,
 '999910799': 0.74,
 '9999107910': 0.89,
 '9999107911': 0.57,
 '9999107912': 0.87,
 '9999107913': 21.77,
 '999910801': 1.19,
 '999910802': 1.42,
 '999910803': 0.22,
 '999910804': 1.04,
 '999910805': 3.79,
 '999910806': 1.49,
 '999910807': 1.1,
 '999910808': 1.19,
 '999910809': 0.36,
 '9999108010': 1.69,
 '9999108011': 2.22,
 '9999108012': 0.26,
 '9999108013': 15.97,
 '999910811': 1.31,
 '999910812': 0.7,
 '999910813': 0.7,
 '999910814': 0.93,
 '999910815': 0.15,
 '999910816': 0.42,
 '999910817': 1.2,
 '999910818': 0.51,
 '999910819': -0.19,
 '9999108110': 1.17,
 '9999108111': -0.32,
 '9999108112': 0.51,
 '9999108113': 7.09,
 '999910821': 4.7,
 '999910822': 0.54,
 '999910823': 0.18,
 '999910824': -0.06,
 '999910825': 0.18,
 '999910826': 0.33,
 '999910827': 1.11,
 '999910828': -0.46,
 '999910829': 0.88,
 '9999108210': 1.24,
 '99991

In [159]:
first_key = list(res['datacontent'].keys())[0]
print(f"{first_key} : {res['datacontent'][first_key]}" )

999910791 : 2.58


In [144]:
res.keys()

dict_keys(['status', 'data-availability', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'metadata', 'datacontent'])

{'999910791': 2.58,
 '999910792': 2.34,
 '999910793': 0.55,
 '999910794': 3.02,
 '999910795': 3.05,
 '999910796': 2.32,
 '999910797': 2.5,
 '999910798': 2.34,
 '999910799': 0.74,
 '9999107910': 0.89,
 '9999107911': 0.57,
 '9999107912': 0.87,
 '9999107913': 21.77,
 '999910801': 1.19,
 '999910802': 1.42,
 '999910803': 0.22,
 '999910804': 1.04,
 '999910805': 3.79,
 '999910806': 1.49,
 '999910807': 1.1,
 '999910808': 1.19,
 '999910809': 0.36,
 '9999108010': 1.69,
 '9999108011': 2.22,
 '9999108012': 0.26,
 '9999108013': 15.97,
 '999910811': 1.31,
 '999910812': 0.7,
 '999910813': 0.7,
 '999910814': 0.93,
 '999910815': 0.15,
 '999910816': 0.42,
 '999910817': 1.2,
 '999910818': 0.51,
 '999910819': -0.19,
 '9999108110': 1.17,
 '9999108111': -0.32,
 '9999108112': 0.51,
 '9999108113': 7.09,
 '999910821': 4.7,
 '999910822': 0.54,
 '999910823': 0.18,
 '999910824': -0.06,
 '999910825': 0.18,
 '999910826': 0.33,
 '999910827': 1.11,
 '999910828': -0.46,
 '999910829': 0.88,
 '9999108210': 1.24,
 '99991

In [138]:
res['metadata']

{'activity': None, 'variable': None}

In [76]:
print(json.dumps(req_json, indent=4))

{
    "status": "OK",
    "data-availability": "available",
    "var": [
        {
            "val": 1,
            "label": "Inflation (Umum)",
            "unit": "",
            "subj": "Consumer Prices Indices",
            "def": "",
            "decimal": "",
            "note": "&lt;p&gt;1. Before April 1979, the base year is September 1966 (September 1966 = 100)&lt;br /&gt;\n2.Since April 1979, the Term &quot;Consumer Price Index&quot; has been used ( the term that has been used before is &quot;Cost Living Index&quot;). The base year is April 1977-Maret 1978. CPI using a consumption pattern obtained from 1977/1978 Cost Living Survey in 17 Provincial Capital cities.) (April 1977-Maret 1978=100).&lt;br /&gt;\n3. Since April 1990-1997, CPI has been based on a consumption pattern obtained from Cost Living Survey in 27 Provincial Capital cities and using 1988/1989 base year) (1988/1989=100)&lt;br /&gt;\n4. Since December 1997, CPI has been based on a consumption pattern obtained fr